# Process Data

## LiDAR Data

In [1]:
import os
import re
import json
import pdal
import rasterio
import numpy as np
from osgeo import gdal

In [ ]:
lidar_dir = "../data/lidar"
dsm_dir = "../data/dsm"
dtm_dir = "../data/dtm"
ndsm_dir = "../data/ndsm"

os.makedirs(dsm_dir, exist_ok=True)
os.makedirs(dtm_dir, exist_ok=True)
os.makedirs(ndsm_dir, exist_ok=True)

def fill_nodata(input_tif, output_tif, max_search_dist=200):
    """
    Fills nodata holes in a raster using GDAL's FillNodata algorithm.
    
    Parameters:
      input_tif (str): Path to the input DTM file with nodata values.
      output_tif (str): Path to the output filled DTM file.
      max_search_dist (int): Maximum search distance in pixels (e.g., 200 for 1 ft resolution to cover 200 ft).
    """
    ds = gdal.Open(input_tif)
    if ds is None:
        raise RuntimeError(f"Could not open input file: {input_tif}")
    band = ds.GetRasterBand(1)
    
    # Create a copy of the raster for output.
    driver = gdal.GetDriverByName('GTiff')
    out_ds = driver.CreateCopy(output_tif, ds, 0)
    out_band = out_ds.GetRasterBand(1)
    
    # Apply FillNodata to interpolate missing values.
    gdal.FillNodata(out_band, None, maxSearchDist=max_search_dist, smoothingIterations=0)
    
    # Close datasets to flush changes.
    out_ds = None
    ds = None

Check crs. https://epsg.io/6565

In [4]:
pipeline_json = {
    "pipeline": [
        "../data/lidar/Philadelphia_100.las",
        {
            "type": "filters.info"
        }
    ]
}

# Run PDAL pipeline.
pipeline = pdal.Pipeline(json.dumps(pipeline_json))
pipeline.execute()

# Retrieve metadata.
metadata = pipeline.metadata

# Extract CRS information from LAS file
crs_wkt = metadata['metadata']['filters.info']['srs']['wkt']

print(crs_wkt)


PROJCS["NAD83(2011) / Pennsylvania South (ftUS)",GEOGCS["NAD83(2011)",DATUM["NAD83_National_Spatial_Reference_System_2011",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],TOWGS84[0,0,0,0,0,0,0],AUTHORITY["EPSG","1116"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["Degree",0.0174532925199433,AUTHORITY["EPSG","9102"]]],PROJECTION["Lambert_Conformal_Conic_2SP"],PARAMETER["false_easting",1968500],PARAMETER["false_northing",0],PARAMETER["central_meridian",-77.75],PARAMETER["standard_parallel_1",40.966666666667],PARAMETER["standard_parallel_2",39.933333333333],PARAMETER["latitude_of_origin",39.333333333333],UNIT["US Survey Foot",0.304800609601219,AUTHORITY["EPSG","9003"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","6565"]]


In [ ]:
# Loop through each LAS file in the lidar directory.
for file in os.listdir(lidar_dir):
    # Match files ending with _<number>.las.
    match = re.search(r'_(\d+)\.las$', file)
    if match:
        num = int(match.group(1))
        # Adjust this condition to process desired files.
        if 600 <= num <= 600:
            file_path = os.path.join(lidar_dir, file)
            base_name = file.split('.')[0]
            
            # Define output filenames.
            output_dsm  = os.path.join(dsm_dir,  f"{base_name}_dsm.tif")
            output_dtm  = os.path.join(dtm_dir,  f"{base_name}_dtm.tif")
            # Create a filename for the filled DTM.
            filled_dtm  = os.path.join(dtm_dir,  f"{base_name}_dtm_filled.tif")
            output_ndsm = os.path.join(ndsm_dir, f"{base_name}_ndsm.tif")

            # ----------------------------------------------------
            # Step 1: Extract bounding box information using PDAL.
            # ----------------------------------------------------
            pipeline_info = {
                "pipeline": [
                    file_path,
                    {"type": "filters.info"}
                ]
            }
            info_pipeline = pdal.Pipeline(json.dumps(pipeline_info))
            info_pipeline.execute()
            metadata = info_pipeline.metadata
            bbox = metadata['metadata']['filters.info']['bbox']
            min_x = bbox['minx']
            max_x = bbox['maxx']
            min_y = bbox['miny']
            max_y = bbox['maxy']

            # ----------------------------------------------------
            # Step 2: Build the DTM pipeline.
            #         - Filter to select only ground points (Class 2)
            #         - Write the DTM using a simple mean aggregation.
            # ----------------------------------------------------
            dtm_pipeline = {
                "pipeline": [
                    file_path,
                    {
                        "type": "filters.range",
                        "limits": "Classification[2:2]"  # Select only bare ground points.
                    },
                    {
                        "type": "writers.gdal",
                        "filename": output_dtm,
                        "output_type": "idw",  
                        "resolution": 1.0,
                        "bounds": f"([{min_x}, {max_x}], [{min_y}, {max_y}])",
                        "override_srs": "EPSG:6565"  # Set the correct CRS.
                    }
                ]
            }
            p_dtm = pdal.Pipeline(json.dumps(dtm_pipeline))
            p_dtm.execute()

            # ----------------------------------------------------
            # Step 3: Build the DSM pipeline.
            #         - Create a DSM using the full point cloud.
            # ----------------------------------------------------
            dsm_pipeline = {
                "pipeline": [
                    file_path,
                    {
                        "type": "writers.gdal",
                        "filename": output_dsm,
                        "output_type": "idw",  # Inverse Distance Weighting interpolation.
                        "resolution": 1.0,
                        "bounds": f"([{min_x}, {max_x}], [{min_y}, {max_y}])",
                        "override_srs": "EPSG:6565"
                    }
                ]
            }
            p_dsm = pdal.Pipeline(json.dumps(dsm_pipeline))
            p_dsm.execute()


Processed file: Philadelphia_600.las


Mosaic the dtm and dsm data.

In [14]:
import os
import subprocess

# Directory containing the dsm  TIFF files
dsm_dir = "../data/dsm"

# Paths for the intermediate VRT and final output TIFF
dsm_vrt_path = "../data/philly/mosaic_dsm.vrt"
dsm_output_path = "../data/philly/mosaic_dsm.tif"

# List all the TIFF files in the directory
tif_files = [os.path.join(dsm_dir, f) for f in os.listdir(dsm_dir) if f.endswith(".tif")]

# Build the VRT mosaic without loading data into memory
# gdalbuildvrt creates a VRT file that references the input files
build_vrt_command = ["gdalbuildvrt", dsm_vrt_path] + tif_files
print("\nRunning command:", " ".join(build_vrt_command))
subprocess.run(build_vrt_command, check=True)

# Optionally, convert the VRT to a GeoTIFF. This step processes the data in chunks.
translate_command = ["gdal_translate", dsm_vrt_path, dsm_output_path]
print("\nRunning command:", " ".join(translate_command))
subprocess.run(translate_command, check=True)

print("\nMosaic created successfully at:", dsm_output_path)

# ---------------------------------------------------- 

# Directory containing the dtm TIFF files
dtm_dir = "../data/dtm"

# Paths for the intermediate VRT and final output TIFF
dtm_vrt_path = "../data/philly/mosaic_dtm.vrt"
dtm_output_path = "../data/philly/mosaic_dtm.tif"

# List all the TIFF files in the directory
tif_files = [os.path.join(dtm_dir, f) for f in os.listdir(dtm_dir) if f.endswith(".tif")]

# Build the VRT mosaic without loading data into memory
# gdalbuildvrt creates a VRT file that references the input files
build_vrt_command = ["gdalbuildvrt", dtm_vrt_path] + tif_files
print("\nRunning command:", " ".join(build_vrt_command))
subprocess.run(build_vrt_command, check=True)

# Optionally, convert the VRT to a GeoTIFF. This step processes the data in chunks.
translate_command = ["gdal_translate", dtm_vrt_path, dtm_output_path]
print("\nRunning command:", " ".join(translate_command))
subprocess.run(translate_command, check=True)

print("\nMosaic created successfully at:", dtm_output_path)


Running command: gdalbuildvrt ../data/philly/mosaic_dsm.vrt ../data/dsm\Philadelphia_100_dsm.tif ../data/dsm\Philadelphia_101_dsm.tif ../data/dsm\Philadelphia_102_dsm.tif ../data/dsm\Philadelphia_103_dsm.tif ../data/dsm\Philadelphia_104_dsm.tif ../data/dsm\Philadelphia_105_dsm.tif ../data/dsm\Philadelphia_106_dsm.tif ../data/dsm\Philadelphia_107_dsm.tif ../data/dsm\Philadelphia_108_dsm.tif ../data/dsm\Philadelphia_109_dsm.tif ../data/dsm\Philadelphia_110_dsm.tif ../data/dsm\Philadelphia_111_dsm.tif ../data/dsm\Philadelphia_112_dsm.tif ../data/dsm\Philadelphia_113_dsm.tif ../data/dsm\Philadelphia_114_dsm.tif ../data/dsm\Philadelphia_115_dsm.tif ../data/dsm\Philadelphia_116_dsm.tif ../data/dsm\Philadelphia_117_dsm.tif ../data/dsm\Philadelphia_118_dsm.tif ../data/dsm\Philadelphia_119_dsm.tif ../data/dsm\Philadelphia_120_dsm.tif ../data/dsm\Philadelphia_121_dsm.tif ../data/dsm\Philadelphia_122_dsm.tif ../data/dsm\Philadelphia_123_dsm.tif ../data/dsm\Philadelphia_124_dsm.tif ../data/dsm\Ph

Then, we can use `fill_nodata` to fill the "holes" in dtm, according to this [tutorial](https://paulojraposo.github.io/pages/PDAL_tutorial.html).

In [2]:
from osgeo import gdal
import numpy as np

def compute_max_from_raster(raster_path):
    ds = gdal.Open(raster_path)
    band = ds.GetRasterBand(1)
    max_val = -np.inf
    block_x, block_y = band.GetBlockSize()
    
    for y in range(0, ds.RasterYSize, block_y):
        rows = block_y if y + block_y < ds.RasterYSize else ds.RasterYSize - y
        for x in range(0, ds.RasterXSize, block_x):
            cols = block_x if x + block_x < ds.RasterXSize else ds.RasterXSize - x
            data = band.ReadAsArray(x, y, cols, rows)
            max_val = max(max_val, np.nanmax(data))
    return max_val

# Usage example:
max_search_dist = compute_max_from_raster("../data/philly/mosaic_dtm.tif")
print("Max search distance (in pixels):", max_search_dist)


Max search distance (in pixels): 447.63400000000007


In [ ]:
def fill_nodata(input_tif, output_tif, max_search_dist=450):
    """
    Fills nodata holes in a raster using GDAL's FillNodata algorithm.
    
    Parameters:
      input_tif (str): Path to the input DTM file with nodata values.
      output_tif (str): Path to the output filled DTM file.
      max_search_dist (int): Maximum search distance in pixels (e.g., 200 for 1 ft resolution to cover 200 ft).
    """
    ds = gdal.Open(input_tif)
    if ds is None:
        raise RuntimeError(f"Could not open input file: {input_tif}")
    band = ds.GetRasterBand(1)
    
    # Create a copy of the raster for output.
    driver = gdal.GetDriverByName('GTiff')
    out_ds = driver.CreateCopy(output_tif, ds, 0)
    out_band = out_ds.GetRasterBand(1)
    
    # Apply FillNodata to interpolate missing values.
    gdal.FillNodata(out_band, None, maxSearchDist=max_search_dist, smoothingIterations=0)
    
    # Close datasets to flush changes.
    out_ds = None
    ds = None

# Create a filled DTM for mosaiced DTM file.
filled_dtm = "../data/philly/mosaic_dtm_filled.tif"
dtm_output_path = "../data/philly/mosaic_dtm.tif"
fill_nodata(dtm_output_path, filled_dtm, max_search_dist=450)

In [4]:
# Create a filled DTM for mosaiced DTM file.
dtm_output_path = "../data/philly/mosaic_dtm_filled.tif"
filled_dtm = "../data/philly/mosaic_dtm_filled_1.tif"
fill_nodata(dtm_output_path, filled_dtm, max_search_dist=1000)

: 

: 

# Building Footprints

In [6]:
import geopandas as gpd
import rasterio
from rasterio.features import rasterize
import rasterio.transform
import matplotlib.pyplot as plt

In [7]:
# Load building footprints
buildings = gpd.read_file("../data/LI_BUILDING_FOOTPRINTS.geojson").to_crs("EPSG:6565")

In [8]:
# Define the raster grid using the total bounds of the reprojected GeoDataFrame
minx, miny, maxx, maxy = buildings.total_bounds
pixel_size = 1  # adjust resolution as needed
width = int((maxx - minx) / pixel_size)
height = int((maxy - miny) / pixel_size)
transform = rasterio.transform.from_origin(minx, maxy, pixel_size, pixel_size)

# Rasterize: burn a value (e.g., 1) for each building footprint
rasterized = rasterize(
    [(geom, 1) for geom in buildings.geometry],
    out_shape=(height, width),
    transform=transform,
    fill=0,
    dtype=rasterio.uint8
)

# Save the rasterized image as a TIF file
with rasterio.open(
    "../data/building_footprints.tif",
    "w",
    driver="GTiff",
    height=height,
    width=width,
    count=1,
    dtype=rasterio.uint8,
    crs="EPSG:6565",
    transform=transform,
) as dst:
    dst.write(rasterized, 1)

Check crs.

In [13]:
with rasterio.open("../data/building_footprints.tif") as src:
    print("CRS:", src.crs)  # Display the CRS

CRS: EPSG:6565


Next step

1. Mosaic the ndsm into one big image
2. cut the ndsm, building raster data, tree raster data into tiles
3. get tree and building dsm for each tile

# NDVI

In [9]:
import os
import numpy as np
import rasterio

input_folder = '../data/cir-naip' # Update with your input folder path
output_folder = '../data/ndvi'# '../data/ndvi'    # Update with your output folder path
os.makedirs(output_folder, exist_ok=True)

for file_name in os.listdir(input_folder):
    # Process only files ending with .tif (case-insensitive)
    if file_name.lower().endswith('.tif'):
        in_file = os.path.join(input_folder, file_name)
        with rasterio.open(in_file) as src:
            red = src.read(1).astype('float32')  # Red band (assumed band 1)
            nir = src.read(4).astype('float32')  # NIR band (assumed band 4)
            ndvi = (nir - red) / (nir + red)
            meta = src.meta.copy()
            meta.update(count=1, dtype='float32')

        # Build output file name by appending _ndvi before the file extension
        base, ext = os.path.splitext(file_name)
        out_file = os.path.join(output_folder, f"{base}_ndvi{ext}")

        with rasterio.open(out_file, 'w', **meta) as dst:
            dst.write(ndvi.astype('float32'), 1)
        
        print(f"Processed {in_file} -> {out_file}")

Processed ../data/cir-naip\pa_m_3907506_ne_18_060_20220517.tif -> ../data/ndvi\pa_m_3907506_ne_18_060_20220517_ndvi.tif
Processed ../data/cir-naip\pa_m_3907506_se_18_060_20220517.tif -> ../data/ndvi\pa_m_3907506_se_18_060_20220517_ndvi.tif
Processed ../data/cir-naip\pa_m_3907507_ne_18_060_20220510.tif -> ../data/ndvi\pa_m_3907507_ne_18_060_20220510_ndvi.tif
Processed ../data/cir-naip\pa_m_3907507_nw_18_060_20220510.tif -> ../data/ndvi\pa_m_3907507_nw_18_060_20220510_ndvi.tif
Processed ../data/cir-naip\pa_m_3907507_se_18_060_20220510.tif -> ../data/ndvi\pa_m_3907507_se_18_060_20220510_ndvi.tif
Processed ../data/cir-naip\pa_m_3907507_sw_18_060_20220510.tif -> ../data/ndvi\pa_m_3907507_sw_18_060_20220510_ndvi.tif
Processed ../data/cir-naip\pa_m_3907508_ne_18_060_20220430.tif -> ../data/ndvi\pa_m_3907508_ne_18_060_20220430_ndvi.tif
Processed ../data/cir-naip\pa_m_3907508_nw_18_060_20220510.tif -> ../data/ndvi\pa_m_3907508_nw_18_060_20220510_ndvi.tif
Processed ../data/cir-naip\pa_m_3907514_

In [1]:
import os
import subprocess
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling

# -------------------------------
# 1. Create a mosaic of NDVI files
# -------------------------------

# Define folders and output file paths
ndvi_folder = '../data/ndvi'
philly_folder = '../data/philly'
vrt_file = os.path.join(ndvi_folder, "mosaic_ndvi.vrt")
mosaic_out_file = os.path.join(ndvi_folder, "mosaic_ndvi.tif")

# List all NDVI files in the folder (matching '_ndvi.tif', case-insensitive)
ndvi_files = [os.path.join(ndvi_folder, f)
              for f in os.listdir(ndvi_folder) if f.lower().endswith('_ndvi.tif')]

# Build the VRT mosaic without loading all data into memory
build_vrt_command = ["gdalbuildvrt", vrt_file] + ndvi_files
print("Running command:", " ".join(build_vrt_command))
subprocess.run(build_vrt_command, check=True)

# Convert the VRT to a GeoTIFF (this processes data in chunks)
translate_command = ["gdal_translate", vrt_file, mosaic_out_file]
print("Running command:", " ".join(translate_command))
subprocess.run(translate_command, check=True)

print("Mosaic NDVI file created at:", mosaic_out_file)

# -------------------------------
# 2. Reproject the mosaic to EPSG:6565
# -------------------------------

# Define the path to the NDSM file (to get the target resolution)
ndsm_path = os.path.join(philly_folder, "mosaic_ndsm.tif")

# Extract the target resolution from the NDSM file (assuming square pixels)
with rasterio.open(ndsm_path) as ndsm_src:
    target_resolution = (abs(ndsm_src.transform.a), abs(ndsm_src.transform.e))
    print("Target resolution from NDSM:", target_resolution)

# Define the target CRS and output file for the reprojected mosaic
dst_crs = 'EPSG:6565'
reprojected_file = os.path.join(philly_folder, "mosaic_ndvi_reprojected.tif")

# Reproject the NDVI mosaic using the target resolution
with rasterio.open(mosaic_out_file) as src:
    # Calculate the new transform, width, and height for the target CRS
    transform, width, height = calculate_default_transform(
        src.crs, dst_crs, src.width, src.height, *src.bounds,
        resolution=target_resolution)
    
    # Update metadata with the new CRS, transform, width, and height
    kwargs = src.meta.copy()
    kwargs.update({
        'crs': dst_crs,
        'transform': transform,
        'width': width,
        'height': height
    })

    # Reproject each band and write the output to a new file
    with rasterio.open(reprojected_file, 'w', **kwargs) as dst:
        for i in range(1, src.count + 1):
            reproject(
                source=rasterio.band(src, i),
                destination=rasterio.band(dst, i),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs=dst_crs,
                resampling=Resampling.nearest)

print("Reprojected NDVI file created at:", reprojected_file)


Running command: gdalbuildvrt ../data/ndvi\mosaic_ndvi.vrt ../data/ndvi\pa_m_3907506_ne_18_060_20220517_ndvi.tif ../data/ndvi\pa_m_3907506_se_18_060_20220517_ndvi.tif ../data/ndvi\pa_m_3907507_ne_18_060_20220510_ndvi.tif ../data/ndvi\pa_m_3907507_nw_18_060_20220510_ndvi.tif ../data/ndvi\pa_m_3907507_se_18_060_20220510_ndvi.tif ../data/ndvi\pa_m_3907507_sw_18_060_20220510_ndvi.tif ../data/ndvi\pa_m_3907508_ne_18_060_20220430_ndvi.tif ../data/ndvi\pa_m_3907508_nw_18_060_20220510_ndvi.tif ../data/ndvi\pa_m_3907514_ne_18_060_20220517_ndvi.tif ../data/ndvi\pa_m_3907515_nw_18_060_20220510_ndvi.tif ../data/ndvi\pa_m_4007449_sw_18_060_20220430_ndvi.tif ../data/ndvi\pa_m_4007457_nw_18_060_20220430_ndvi.tif ../data/ndvi\pa_m_4007457_sw_18_060_20220430_ndvi.tif ../data/ndvi\pa_m_4007554_se_18_060_20220517_ndvi.tif ../data/ndvi\pa_m_4007555_se_18_060_20220510_ndvi.tif ../data/ndvi\pa_m_4007555_sw_18_060_20220510_ndvi.tif ../data/ndvi\pa_m_4007556_se_18_060_20220430_ndvi.tif ../data/ndvi\pa_m_40075

Check crs.

In [3]:
with rasterio.open(reprojected_file) as src:
    print("CRS:", src.crs)  # Display the CRS
    print("Resolution:", (src.res[0], src.res[1]))  # Display the resolution

CRS: EPSG:6565
Resolution: (1.0, 1.0)


## Clip into tiles

In [ ]:
import os
import math
import rasterio
from rasterio.windows import from_bounds, Window
from rasterio.transform import Affine

# File paths
dsm_path = "../data/philly/mosaic_dsm.tif"
dtm_path = "../data/philly/mosaic_dtm.tif"
ndvi_path = "../data/philly/mosaic_ndvi_reprojected.tif"
buildings_path = "../data/philly/building_footprints.tif"

# List of input file paths and corresponding output folders
files = [
    {"path": dsm_path, "name": "dsm"},
    {"path": dtm_path, "name": "dtm"},
    {"path": ndvi_path, "name": "ndvi"},
    {"path": buildings_path, "name": "buildings"}
]

# Open all datasets to obtain their bounds
datasets = [rasterio.open(f["path"]) for f in files]

# Open the building footprints dataset to get its bounds
with rasterio.open(buildings_path) as bldg_ds:
    bldg_bounds = bldg_ds.bounds

# Use the building footprints bounds as the common bounds
left = bldg_bounds.left
bottom = bldg_bounds.bottom
right = bldg_bounds.right
top = bldg_bounds.top

print(f"Common bounds (from building footprints): left={left}, bottom={bottom}, right={right}, top={top}")

# Define tile size in pixels
tile_size = 10000

# Create an output folder if it does not exist
output_dir = "../data/tiles"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

def process_file(ds, file_info):
    # Compute the window in the dataset that corresponds to the common bounds
    window = from_bounds(left, bottom, right, top, transform=ds.transform)
    
    # Calculate window dimensions in pixels
    w_width = int(window.width)
    w_height = int(window.height)
    print(f"{file_info['name']} window size: {w_width}x{w_height}")

    # Define output subfolder for this file
    out_subfolder = os.path.join(output_dir, file_info["name"])
    if not os.path.exists(out_subfolder):
        os.makedirs(out_subfolder)
    
    # Calculate the number of tiles required in each dimension (round up)
    n_tiles_x = math.ceil(w_width / tile_size)
    n_tiles_y = math.ceil(w_height / tile_size)
    
    tile_id = 0
    for i in range(n_tiles_y):
        for j in range(n_tiles_x):
            # Calculate offsets in the window (in pixels)
            row_off = i * tile_size
            col_off = j * tile_size

            # Create a window relative to the original dataset with full tile size.
            tile_window = Window(
                col_off=window.col_off + col_off,
                row_off=window.row_off + row_off,
                width=tile_size,
                height=tile_size
            )
            
            # Read the tile data with boundless=True to pad missing areas with nodata.
            nodata_value = ds.nodata if ds.nodata is not None else 0
            tile_data = ds.read(window=tile_window, boundless=True, fill_value=nodata_value)

            # Compute the transform for the tile
            tile_transform = rasterio.windows.transform(tile_window, ds.transform)

            # Update metadata for the tile
            out_meta = ds.meta.copy()
            out_meta.update({
                "height": tile_size,
                "width": tile_size,
                "transform": tile_transform
            })

            # Save the tile
            out_filename = os.path.join(out_subfolder, f"{file_info['name']}_tile_{tile_id}.tif")
            with rasterio.open(out_filename, "w", **out_meta) as dest:
                dest.write(tile_data)
            tile_id += 1
    print(f"Saved {tile_id} tiles for {file_info['name']}.")

# Process each file
for file_info in files:
    with rasterio.open(file_info["path"]) as ds:
        process_file(ds, file_info)

# Close all opened datasets
for ds in datasets:
    ds.close()

print("Tiling complete.")


Common bounds (from building footprints): left=2660825.867176956, bottom=206275.7414628868, right=2749675.867176956, top=304888.7414628868
dsm window size: 88850x98613


KeyboardInterrupt: 

In [ ]:

            # ----------------------------------------------------
            # Step 4: Compute the nDSM (normalized DSM).
            #         nDSM = DSM - filled DTM
            # ----------------------------------------------------
            with rasterio.open(output_dsm) as dsm_src, rasterio.open(filled_dtm) as dtm_src:
                dsm_data = dsm_src.read(1)
                dtm_data = dtm_src.read(1)
                ndsm_data = dsm_data - dtm_data

                # Update metadata for the nDSM output.
                ndsm_meta = dsm_src.meta.copy()
                ndsm_meta.update({
                    "dtype": "float32",
                    "crs": "EPSG:6565"
                })

                with rasterio.open(output_ndsm, "w", **ndsm_meta) as dst:
                    dst.write(ndsm_data.astype(np.float32), 1)

            print(f"Processed file: {file}")


Then, mosaic all the tiles into one raster.

In [2]:
import os
import subprocess

# Directory containing the NDSM TIFF files
ndsm_dir = "../data/ndsm"
# Paths for the intermediate VRT and final output TIFF
vrt_path = "../data/philly/mosaic_ndsm.vrt"
output_path = "../data/philly/mosaic_ndsm.tif"

# List all the TIFF files in the directory
tif_files = [os.path.join(ndsm_dir, f) for f in os.listdir(ndsm_dir) if f.endswith(".tif")]

# Build the VRT mosaic without loading data into memory
# gdalbuildvrt creates a VRT file that references the input files
build_vrt_command = ["gdalbuildvrt", vrt_path] + tif_files
print("\nRunning command:", " ".join(build_vrt_command))
subprocess.run(build_vrt_command, check=True)

# Optionally, convert the VRT to a GeoTIFF. This step processes the data in chunks.
translate_command = ["gdal_translate", vrt_path, output_path]
print("\nRunning command:", " ".join(translate_command))
subprocess.run(translate_command, check=True)

print("\nMosaic created successfully at:", output_path)



Running command: gdalbuildvrt ../data/philly/mosaic_ndsm.vrt ../data/ndsm\Philadelphia_100_ndsm.tif ../data/ndsm\Philadelphia_101_ndsm.tif ../data/ndsm\Philadelphia_102_ndsm.tif ../data/ndsm\Philadelphia_103_ndsm.tif ../data/ndsm\Philadelphia_104_ndsm.tif ../data/ndsm\Philadelphia_105_ndsm.tif ../data/ndsm\Philadelphia_106_ndsm.tif ../data/ndsm\Philadelphia_107_ndsm.tif ../data/ndsm\Philadelphia_108_ndsm.tif ../data/ndsm\Philadelphia_109_ndsm.tif ../data/ndsm\Philadelphia_110_ndsm.tif ../data/ndsm\Philadelphia_111_ndsm.tif ../data/ndsm\Philadelphia_112_ndsm.tif ../data/ndsm\Philadelphia_113_ndsm.tif ../data/ndsm\Philadelphia_114_ndsm.tif ../data/ndsm\Philadelphia_115_ndsm.tif ../data/ndsm\Philadelphia_116_ndsm.tif ../data/ndsm\Philadelphia_117_ndsm.tif ../data/ndsm\Philadelphia_118_ndsm.tif ../data/ndsm\Philadelphia_119_ndsm.tif ../data/ndsm\Philadelphia_120_ndsm.tif ../data/ndsm\Philadelphia_121_ndsm.tif ../data/ndsm\Philadelphia_122_ndsm.tif ../data/ndsm\Philadelphia_123_ndsm.tif ..

## Mask

In [7]:
import os
import rasterio
import numpy as np
from rasterio.windows import from_bounds
import rasterio.warp

# Directories for the input tiled datasets (created previously)
tiles_dir = "../data/tiles"
ndsm_tiles_dir = os.path.join(tiles_dir, "ndsm")
ndvi_tiles_dir = os.path.join(tiles_dir, "ndvi")
buildings_tiles_dir = os.path.join(tiles_dir, "buildings")

# Output directories for DSM products
output_base = "../data/tiles_output"
os.makedirs(output_base, exist_ok=True)

tree_dsm_dir = os.path.join(output_base, "tree_dsm")
bldg_dsm_dir = os.path.join(output_base, "building_dsm")

# Create output directories if they do not exist
os.makedirs(tree_dsm_dir, exist_ok=True)
os.makedirs(bldg_dsm_dir, exist_ok=True)

# NDVI threshold for tree mask and a default nodata value
ndvi_threshold = 0.5
default_nodata = -9999

# Get list of NDSM tile files (assuming a naming pattern like "ndsm_tile_*.tif")
ndsm_tile_files = [f for f in os.listdir(ndsm_tiles_dir) if f.endswith(".tif")]

print(f"Found {len(ndsm_tile_files)} NDSM tiles. Processing each tile...")


Found 90 NDSM tiles. Processing each tile...


In [8]:
for tile_file in ndsm_tile_files:
    # Derive corresponding file paths for NDVI and building footprints tiles.
    # Here we assume that the tiles have the same file name for each dataset.
    ndsm_tile_path = os.path.join(ndsm_tiles_dir, tile_file)
    ndvi_tile_path = os.path.join(ndvi_tiles_dir, tile_file.replace("ndsm_tile", "ndvi_tile"))
    buildings_tile_path = os.path.join(buildings_tiles_dir, tile_file.replace("ndsm_tile", "buildings_tile"))
    
    print(f"\nProcessing tile: {tile_file}")
    
    # Open the NDSM tile and extract its bounds, data, metadata, and nodata value.
    with rasterio.open(ndsm_tile_path) as ndsm_src:
        ndsm_bounds = ndsm_src.bounds
        ndsm_data = ndsm_src.read(1)  # assumed single band
        ndsm_meta = ndsm_src.meta.copy()
        ndsm_nodata = ndsm_src.nodata if ndsm_src.nodata is not None else default_nodata
        ndsm_meta.update(nodata=ndsm_nodata)
    
    # Since the tile extents are identical, we can read the NDVI and building data directly.
    with rasterio.open(ndvi_tile_path) as ndvi_src:
        ndvi_data = ndvi_src.read(1)
        print(f"NDVI tile shape: {ndvi_data.shape}")
    
    with rasterio.open(buildings_tile_path) as bldg_src:
        buildings_data = bldg_src.read(1)
        print(f"Building tile shape: {buildings_data.shape}")
    
    # Create tree mask using NDVI threshold
    tree_mask = ndvi_data >= ndvi_threshold
    tree_pixels = np.sum(tree_mask)
    print(f"Tree pixels: {tree_pixels}")
    
    # Create tree DSM by masking the NDSM with the tree mask
    tree_dsm = np.copy(ndsm_data)
    tree_dsm[~tree_mask] = ndsm_nodata
    
    # Create building mask (assuming building footprints are indicated with positive values)
    buildings_mask = buildings_data > 0
    bldg_pixels = np.sum(buildings_mask)
    print(f"Building pixels: {bldg_pixels}")
    
    building_dsm = np.copy(ndsm_data)
    building_dsm[~buildings_mask] = ndsm_nodata
    
    # Prepare output filenames using the tile base name
    tile_base = os.path.splitext(tile_file)[0]  # e.g., "ndsm_tile_0"
    tree_out_path = os.path.join(tree_dsm_dir, f"{tile_base}_tree_dsm.tif")
    bldg_out_path = os.path.join(bldg_dsm_dir, f"{tile_base}_bldg_dsm.tif")
    
    # Write the tree DSM tile
    print(f"Writing tree DSM to {tree_out_path}")
    with rasterio.open(tree_out_path, 'w', **ndsm_meta) as tree_dst:
        tree_dst.write(tree_dsm, 1)
    
    # Write the building DSM tile
    print(f"Writing building DSM to {bldg_out_path}")
    with rasterio.open(bldg_out_path, 'w', **ndsm_meta) as bldg_dst:
        bldg_dst.write(building_dsm, 1)

print("\nProcessing complete for all tiles!")


Processing tile: ndsm_tile_0.tif
NDVI tile shape: (10000, 10000)
Building tile shape: (10000, 10000)
Tree pixels: 10482600
Building pixels: 0
Writing tree DSM to ../data/tiles_output\tree_dsm\ndsm_tile_0_tree_dsm.tif
Writing building DSM to ../data/tiles_output\building_dsm\ndsm_tile_0_bldg_dsm.tif

Processing tile: ndsm_tile_1.tif
NDVI tile shape: (10000, 10000)
Building tile shape: (10000, 10000)
Tree pixels: 11177410
Building pixels: 0
Writing tree DSM to ../data/tiles_output\tree_dsm\ndsm_tile_1_tree_dsm.tif
Writing building DSM to ../data/tiles_output\building_dsm\ndsm_tile_1_bldg_dsm.tif

Processing tile: ndsm_tile_10.tif
NDVI tile shape: (10000, 10000)
Building tile shape: (10000, 10000)
Tree pixels: 12064561
Building pixels: 116566
Writing tree DSM to ../data/tiles_output\tree_dsm\ndsm_tile_10_tree_dsm.tif
Writing building DSM to ../data/tiles_output\building_dsm\ndsm_tile_10_bldg_dsm.tif

Processing tile: ndsm_tile_11.tif
NDVI tile shape: (10000, 10000)
Building tile shape: (

Mosaic into tree dsm and building dsm.

In [9]:
import os
import subprocess

# -------------------------------
# Mosaic Tree DSM
# -------------------------------

# Directory containing the tree DSM tiles
tree_dsm_dir = "../data/tiles_output/tree_dsm"
# Paths for the intermediate VRT and final output TIFF for tree DSM
tree_vrt_path = "../data/philly/mosaic_tree_dsm.vrt"
tree_output_path = "../data/philly/mosaic_tree_dsm.tif"

# List all the tree DSM TIFF files in the directory
tree_tif_files = [os.path.join(tree_dsm_dir, f) for f in os.listdir(tree_dsm_dir) if f.endswith(".tif")]

# Build the VRT mosaic without loading data into memory for tree DSM
build_vrt_command = ["gdalbuildvrt", tree_vrt_path] + tree_tif_files
print("\nRunning command:", " ".join(build_vrt_command))
subprocess.run(build_vrt_command, check=True)

# Convert the VRT to a GeoTIFF for tree DSM (this processes the data in chunks)
translate_command = ["gdal_translate", tree_vrt_path, tree_output_path]
print("\nRunning command:", " ".join(translate_command))
subprocess.run(translate_command, check=True)

print("\nTree DSM mosaic created successfully at:", tree_output_path)


# -------------------------------
# Mosaic Building DSM
# -------------------------------

# Directory containing the building DSM tiles
bldg_dsm_dir = "../data/tiles_output/building_dsm"
# Paths for the intermediate VRT and final output TIFF for building DSM
bldg_vrt_path = "../data/philly/mosaic_building_dsm.vrt"
bldg_output_path = "../data/philly/mosaic_building_dsm.tif"

# List all the building DSM TIFF files in the directory
bldg_tif_files = [os.path.join(bldg_dsm_dir, f) for f in os.listdir(bldg_dsm_dir) if f.endswith(".tif")]

# Build the VRT mosaic without loading data into memory for building DSM
build_vrt_command = ["gdalbuildvrt", bldg_vrt_path] + bldg_tif_files
print("\nRunning command:", " ".join(build_vrt_command))
subprocess.run(build_vrt_command, check=True)

# Convert the VRT to a GeoTIFF for building DSM
translate_command = ["gdal_translate", bldg_vrt_path, bldg_output_path]
print("\nRunning command:", " ".join(translate_command))
subprocess.run(translate_command, check=True)

print("\nBuilding DSM mosaic created successfully at:", bldg_output_path)



Running command: gdalbuildvrt ../data/philly/mosaic_tree_dsm.vrt ../data/tiles_output/tree_dsm\ndsm_tile_0_tree_dsm.tif ../data/tiles_output/tree_dsm\ndsm_tile_10_tree_dsm.tif ../data/tiles_output/tree_dsm\ndsm_tile_11_tree_dsm.tif ../data/tiles_output/tree_dsm\ndsm_tile_12_tree_dsm.tif ../data/tiles_output/tree_dsm\ndsm_tile_13_tree_dsm.tif ../data/tiles_output/tree_dsm\ndsm_tile_14_tree_dsm.tif ../data/tiles_output/tree_dsm\ndsm_tile_15_tree_dsm.tif ../data/tiles_output/tree_dsm\ndsm_tile_16_tree_dsm.tif ../data/tiles_output/tree_dsm\ndsm_tile_17_tree_dsm.tif ../data/tiles_output/tree_dsm\ndsm_tile_18_tree_dsm.tif ../data/tiles_output/tree_dsm\ndsm_tile_19_tree_dsm.tif ../data/tiles_output/tree_dsm\ndsm_tile_1_tree_dsm.tif ../data/tiles_output/tree_dsm\ndsm_tile_20_tree_dsm.tif ../data/tiles_output/tree_dsm\ndsm_tile_21_tree_dsm.tif ../data/tiles_output/tree_dsm\ndsm_tile_22_tree_dsm.tif ../data/tiles_output/tree_dsm\ndsm_tile_23_tree_dsm.tif ../data/tiles_output/tree_dsm\ndsm_tile_

In [10]:
import rasterio
import matplotlib.pyplot as plt
import numpy as np

def visualize_and_export(tif_path, png_path, scale=0.1):
    """
    Visualize a large GeoTIFF by downsampling it and export the result as a PNG.
    
    Parameters:
        tif_path (str): Path to the input GeoTIFF.
        png_path (str): Path for the output PNG file.
        scale (float): Factor by which to downsample the image. 
                       0.1 means 10% of the original dimensions.
    """
    with rasterio.open(tif_path) as src:
        # Get original dimensions
        orig_height = src.height
        orig_width = src.width
        print(f"Original shape for {tif_path}: {orig_width}x{orig_height}")
        
        # Calculate new dimensions based on the scale factor
        new_height = int(orig_height * scale)
        new_width = int(orig_width * scale)
        print(f"Resampling to {new_width}x{new_height}")
        
        # Read the first band and resample on the fly
        data = src.read(
            1,
            out_shape=(1, new_height, new_width),
            resampling=rasterio.enums.Resampling.nearest
        )
        # Remove the extra dimension
        data = data.squeeze()
    
    # Plot the data using a colormap (adjust cmap if needed)
    plt.figure(figsize=(10, 10))
    plt.imshow(data, cmap='viridis')
    plt.colorbar(label='DSM value')
    plt.title(tif_path)
    plt.axis('off')
    
    # Save the figure as a PNG file
    plt.savefig(png_path, bbox_inches='tight', dpi=300)
    plt.close()
    print(f"Saved PNG: {png_path}")

# File paths for the mosaicked DSMs
tree_dsm_tif = "../data/philly/mosaic_tree_dsm.tif"
bldg_dsm_tif = "../data/philly/mosaic_building_dsm.tif"

# Output PNG file paths
tree_dsm_png = "../data/philly/mosaic_tree_dsm.png"
bldg_dsm_png = "../data/philly/mosaic_building_dsm.png"

# Visualize and export each DSM as a PNG.
visualize_and_export(tree_dsm_tif, tree_dsm_png, scale=0.1)
visualize_and_export(bldg_dsm_tif, bldg_dsm_png, scale=0.1)


Original shape for ../data/philly/mosaic_tree_dsm.tif: 90000x100000
Resampling to 9000x10000
Saved PNG: ../data/philly/mosaic_tree_dsm.png
Original shape for ../data/philly/mosaic_building_dsm.tif: 90000x100000
Resampling to 9000x10000
Saved PNG: ../data/philly/mosaic_building_dsm.png
